### Create Metrics

Apply logic from doc in `docs`

In [1]:
import pandas as pd

In [80]:
joined_data = pd.read_csv('../data/cleaned_prep.csv')

#### Product Range

In [81]:
# Waste qty > 0 on days when ranging indicator is 0
joined_data['product_waste'] = (joined_data['waste_value_on_day'] > 0) & (joined_data['ranging_indicator_on_day'] == 0)

In [82]:
# Stockout indicator 1 on a day when ranging indicator is 0
joined_data['product_stockout'] = (joined_data['stock_out_ind_on_day'] == 1) & (joined_data['ranging_indicator_on_day'] == 0)

#### Depot Inaccurate

In [83]:
# Received_units (store) > Allocation (store) Shelf life before and including waste event
joined_data['depot_waste'] = (joined_data['delivered_qty_over_shelf_life'] > joined_data['allocated_qty_over_shelf_life'])

In [84]:
# Received_units (store) < Allocation (store) Allocation to store lead time before and including stockout
joined_data['depot_stockout'] = (joined_data['delivered_qty_over_shelf_life'] < joined_data['allocated_required_diff_high_qty_over_lead_time'])

In [85]:
(joined_data['depot_waste'] & joined_data['depot_stockout']).value_counts()

False    1749092
True        1773
dtype: int64

#### Sales Forecast Error

In [86]:
### Sales - Note: missing lost sales
## Add in thresholds
# Sales Forecast Error
joined_data['forecast_waste'] = (
    0.75*joined_data['forecast_demand_qty_over_shelf_life'] > joined_data['sales_qty_over_shelf_life']
    ) & (
    (joined_data['forecast_demand_qty_over_shelf_life'] - joined_data['sales_qty_over_shelf_life']).abs() > 1
)


# Forecast Stockout
joined_data['forecast_stockout'] = (
    1.25*joined_data['forecast_demand_qty_over_lead_time'] < joined_data['sales_qty_over_lead_time'])  & (
    (joined_data['forecast_demand_qty_over_lead_time']- joined_data['sales_qty_over_lead_time']).abs() > 1
)

In [87]:
(joined_data['forecast_stockout'] & joined_data['forecast_waste']).value_counts()

False    1735767
True       15098
dtype: int64

#### Waste Prediction Error

In [88]:
### Waste  - add in threshold
joined_data['waste_prediction_waste'] = (
    0.75*joined_data['forecast_waste_qty_over_shelf_life'] > joined_data['waste_value_on_day']
)

In [89]:
### Stockout - add in threshold
joined_data['waste_prediction_stockout'] = (
    1.25*joined_data['forecast_waste_qty_over_lead_time'] < (joined_data['waste_value_on_day']) 
)

In [90]:
(joined_data['waste_prediction_waste'] & joined_data['waste_prediction_stockout']).value_counts()

False    1726660
True       24205
dtype: int64

#### Stockfile Adjustment

In [91]:
### Stock
# Stock file adjustment
joined_data['stock_waste'] = (joined_data['stockfile_adjust_qty_on_day'] > 0) & (joined_data['stockfile_adjust_qty_at_plus_1_day'] > 0)

# Stock had less stock than reported
joined_data['stock_stockout'] = (joined_data['stockfile_adjust_qty_on_day'] < 0) & (
joined_data['stockfile_adjust_qty_at_minus_1_day'] < 0)

In [92]:
assert any(joined_data['stock_waste'] & joined_data['stock_stockout']) == False

#### Negative Stockfile

In [93]:
### Negative stockfile
    
joined_data['negative_stockfile'] = (joined_data['closing_inventory_on_day'] < 0) # missing day before data 

### Review Metrics

In [94]:
metrics = ['product_waste', 'product_stockout', 'depot_waste', 'depot_stockout', 
           'forecast_waste', 'forecast_stockout', 'waste_prediction_waste',
          'waste_prediction_stockout', 'stock_waste', 'stock_stockout', 'negative_stockfile']

In [95]:
(joined_data[metrics].apply(lambda x: x.value_counts())
                     .transpose()
                     .sort_values(True, ascending=False))

,False,True
waste_prediction_waste,659932.0,1090933.0
forecast_stockout,1316657.0,434208.0
forecast_waste,1415386.0,335479.0
waste_prediction_stockout,1475008.0,275857.0
depot_stockout,1648337.0,102528.0
depot_waste,1706111.0,44754.0
stock_stockout,1750342.0,523.0
product_waste,1750583.0,282.0
stock_waste,1750635.0,230.0
product_stockout,1750865.0,NaN


In [98]:
waste_slice = joined_data[joined_data['waste_prediction_waste']]

In [99]:
waste_slice['forecast_waste_qty_over_lead_time']

1          0.06
2          0.01
3          0.45
4          2.37
7          0.07
           ... 
1750860    0.59
1750861    1.67
1750862    0.30
1750863    2.52
1750864    0.96
Name: forecast_waste_qty_over_lead_time, Length: 1090933, dtype: float64

In [101]:
waste_slice['waste_value_on_day'].value_counts()

0.00      1042766
1.00         3606
4.00         3187
3.00         3012
2.00         2839
           ...   
23.80           1
176.00          1
96.00           1
6.83            1
0.26            1
Name: waste_value_on_day, Length: 895, dtype: int64